In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url="https://www.transfermarkt.fr/ligue-1/transfers/wettbewerb/FR1"

headers = {
    'User-Agent': 'Mozilla'
}
request_text = requests.get(url,headers=headers)

In [3]:
request_text.status_code

200

In [4]:
page = BeautifulSoup(request_text.content, "lxml")

tables=page.find_all('div',class_='responsive-table')

In [5]:
def create_table(team,table):
    """ cette fonction permet de prender un tableau en entrée sous la forme de code html et après
    elle le transforme en un vrai tableau, cette fonction retourne aussi l'état du tableau: 
     càd est ce que le tableau est remplie de joueurs arrivant au team ou ce sont des joueurs vendus par le team
       """


    lignes=table.find_all('tr')
    list0=[]
    ligne0=lignes[0]
    colonnes0=ligne0.find_all('th')
    for colonne in colonnes0:
        list0.append(colonne.text)
    
    etat=list0[0]
    taille=len(lignes)-1
    table=[0 for i in range(taille)]
    colonnes=[0 for i in range(taille)]
    row=[0 for i in range(taille)]
    for i in range(1,len(lignes)):
        table[i-1]=lignes[i]
        colonnes[i-1]=table[i-1].find_all('td')
        row[i-1]=[]
        for colonne in colonnes[i-1]:
            title=colonne.find('img')
            if title!=None:
                row[i-1].append(title.get("title"))
            else:
                row[i-1].append(colonne.text)
                
    df=pd.DataFrame(row)

    return df,etat

In [6]:
table=tables[0]
df0, etat=create_table('walu',table)

In [8]:
df0.head()

,0,1,2,3,4,5,6,7,8
0,Randal Kolo MuaniR. Kolo Muani,24,France,Avant-centre,AC,"80,00 mio. €",Eintracht Francfort,Allemagne,"95,00 mio. €"
1,Manuel UgarteM. Ugarte,22,Uruguay,Milieu défensif,MDC,"55,00 mio. €",Sporting CP,Portugal,"60,00 mio. €"
2,Ousmane DembéléO. Dembélé,26,France,Ailier droit,AiD,"60,00 mio. €",FC Barcelone,Espagne,"50,00 mio. €"
3,Bradley BarcolaB. Barcola,20,France,Ailier droit,AiD,"35,00 mio. €",Olympique Lyonnais,France,"45,00 mio. €"
4,Lucas HernándezL. Hernández,27,France,Défenseur central,DC,"45,00 mio. €",Bayern Munich,Allemagne,"45,00 mio. €"


In [10]:
infos_equipe=page.find_all('div',class_='box')
all_teams=[]
for info in infos_equipe:
    team=info.find('a')
    if team!=None:
        team=team.get('title')
        all_teams.append(team)
teams=[team for team in all_teams if team!=None]
print(teams)


['Paris Saint-Germain', 'RC Lens', 'Olympique de Marseille', 'AS Monaco', 'Stade Rennais FC', 'LOSC Lille', 'Olympique Lyonnais', 'Clermont Foot 63', 'OGC Nice', 'FC Lorient', 'Stade de Reims', 'Montpellier Hérault SC', 'Toulouse FC', 'Stade Brestois 29', 'RC Strasbourg Alsace', 'FC Nantes', 'Le Havre AC', 'FC Metz', 'Rayan Cherki']


In [11]:
df=[0 for i in range(len(tables))]

for i in range(len(tables)):
    table=tables[i]
   
    team=teams[(i//2)]
    
    df[i],etat=create_table(team,table)

    if etat=="Arrivées":
        df[i]['team_acheteur']=team
    else:
        df[i]['team vendeur']=team
         

In [12]:
concatenated_df = pd.concat(df, ignore_index=True)

In [13]:
concatenated_df.head()

,0,1,2,3,4,5,6,7,8,team_acheteur,team vendeur
0,Randal Kolo MuaniR. Kolo Muani,24,France,Avant-centre,AC,"80,00 mio. €",Eintracht Francfort,Allemagne,"95,00 mio. €",Paris Saint-Germain,NaN
1,Manuel UgarteM. Ugarte,22,Uruguay,Milieu défensif,MDC,"55,00 mio. €",Sporting CP,Portugal,"60,00 mio. €",Paris Saint-Germain,NaN
2,Ousmane DembéléO. Dembélé,26,France,Ailier droit,AiD,"60,00 mio. €",FC Barcelone,Espagne,"50,00 mio. €",Paris Saint-Germain,NaN
3,Bradley BarcolaB. Barcola,20,France,Ailier droit,AiD,"35,00 mio. €",Olympique Lyonnais,France,"45,00 mio. €",Paris Saint-Germain,NaN
4,Lucas HernándezL. Hernández,27,France,Défenseur central,DC,"45,00 mio. €",Bayern Munich,Allemagne,"45,00 mio. €",Paris Saint-Germain,NaN
